# Calling Kamodo Directly

Demonstration for how to call the Kamodo models directly.  This uses the SingleSatellite method developed by RR.

## Test the accuracy of the TIEGCM Kamodo Reader

### Functions for reading TIEGCM manually:

In [1]:
from netCDF4 import Dataset
import numpy as np 
import os
import warnings
import pandas as pd
from datetime import datetime, timezone, timedelta

warnings.filterwarnings("ignore")


def ncdump(path , verb=True):
    
    nc_fid = Dataset(path)

    
    '''
    ncdump outputs dimensions, variables and their attribute information.
    The information is similar to that of NCAR's ncdump utility.
    ncdump requires a valid instance of Dataset.

    Parameters
    ----------
    nc_fid : netCDF4.Dataset
        A netCDF4 dateset object
    verb : Boolean
        whether or not nc_attrs, nc_dims, and nc_vars are printed

    Returns
    -------
    nc_attrs : list
        A Python list of the NetCDF file global attributes
    nc_dims : list
        A Python list of the NetCDF file dimensions
    nc_vars : list
        A Python list of the NetCDF file variables
    '''
    def print_ncattr(key):
        """
        Prints the NetCDF file attributes for a given key

        Parameters
        ----------
        key : unicode
            a valid netCDF4.Dataset.variables key
        """
        try:
            print("\t\ttype:", repr(nc_fid.variables[key].dtype))
            for ncattr in nc_fid.variables[key].ncattrs():
                print('\t\t%s:' % ncattr)
                repr(nc_fid.variables[key].getncattr(ncattr))
        except KeyError:
            print("\t\tWARNING: %s does not contain variable attributes" % key)

    # NetCDF global attributes
    nc_attrs = nc_fid.ncattrs()
    if verb:
        print("NetCDF Global Attributes:")
        for nc_attr in nc_attrs:
            print('\t%s:' % nc_attr, repr(nc_fid.getncattr(nc_attr)))
    nc_dims = [dim for dim in nc_fid.dimensions]  # list of nc dimensions
    # Dimension shape information.
    if verb:
        print("NetCDF dimension information:")
        for dim in nc_dims:
            print("\tName:", dim )
            print("\t\tsize:", len(nc_fid.dimensions[dim]))
            print_ncattr(dim)
    # Variable information.
    nc_vars = [var for var in nc_fid.variables]  # list of nc variables
    if verb:
        print("NetCDF variable information:")
        for var in nc_vars:
            if var not in nc_dims:
                print('\tName:', var)
                print("\t\tdimensions:", nc_fid.variables[var].dimensions)
                print("\t\tsize:", nc_fid.variables[var].size)
                print_ncattr(var)
    return(nc_attrs, nc_dims, nc_vars)





def read_tiegcm( filename, variables):
    ''' This function reads the TIEGCM .nc files and saves the given input variables to a dictionary.
        The breakloop feature is here so that if the file doesn't exist the code can still continue if placed in a loop.'''
    status = os.path.exists(filename)
 
    if status == True:
        data = {}
        for i, var_names in enumerate(variables):
            ncid =  Dataset(filename,"r+", format="NETCDF4")# filename must be a string
            varData = ncid.variables
            data[var_names] = np.array(varData[var_names])  
    elif status == False:
        print('No File Found', filename )
        breakloop = True
        data = 0
        return( data , breakloop)
    breakloop = False
    return(data,breakloop )


def find_modelgrid_base_index(TIEGCM, lon_sat, lat_sat, time_sat):
    """ 
    Interpolation function: (3 / 5)

    This function locates the grid points that make up thr square surrounding out desired point.
    """

    lon_start = -180 # first longitude point of the model grid (deg)
    dlon = 5  # difference between adjacent longitude grid points (deg)
    index_lon0 = int(np.mod(np.floor( (lon_sat - lon_start )/dlon ), 72 ) ) 


    lat_start = -87.5 # first longitude point of the model grid (deg)
    dlat = 5 # difference between adjacent longitude grid points (deg)
    index_lat0 = int(np.mod(np.floor( (lat_sat - lat_start)/dlat ), 36 ) )

    time_start = 0 # first longitude point of the model grid (deg)
    dtime = 1 # difference between adjacent longitude grid points (deg)
    index_time = int((np.mod(np.floor( (time_sat - time_start)/dtime ), 24 ) )) -1

    if index_lon0 == 71:
        index_lon1 = 0
    else:
        index_lon1 = index_lon0 + 1


    if index_lat0 >= 35 :
        index_lat1 = 35
        NPole_Flag = True
        SPole_Flag = False 

    elif index_lat0 <= 0: 
        index_lat1 = 0
        SPole_Flag = True
        NPole_Flag = False
    else: 
        index_lat1 = index_lat0 + 1
        NPole_Flag = False 
        SPole_Flag = False 


    return(index_lon0, index_lat0, index_lon1, index_lat1, index_time, NPole_Flag, SPole_Flag )


def DEN_and_ZG_vertprofs(TIEGCM, lon_sat, lat_sat, time_sat, param):
    '''
    Interpolation function: (2 / 5)

    This function finds the vertical profiles of the four grid points around a desired point.
    '''
    
    #### First we find the four grid points:
    (indexlon0, 
    indexlat0, 
    indexlon1, 
    indexlat1, 
    indexut, 
    NPole_Flag, 
    SPole_Flag) = find_modelgrid_base_index(TIEGCM, lon_sat, lat_sat, time_sat)
    grid_vals = pd.DataFrame(data = {'lon0' : TIEGCM['lon'][indexlon0] ,\
                                 'lonindex0': indexlon0 ,\
                                 'lon1'     :TIEGCM['lon'][indexlon1] ,\
                                 'lonindex1':indexlon1 ,\
                                 'lat0'     :TIEGCM['lat'][indexlat0] ,\
                                 'latindex0':indexlat0 ,\
                                 'lat1'     :TIEGCM['lat'][indexlat1] ,\
                                 'latindex1':indexlat1  }, index=[0])
#     print('UT index',indexut)
#     print('UT value',TIEGCM['ut'][indexut])
#     print()
#     print('Lon0 index',indexlon0)
#     print('Lon0 value',TIEGCM['lon'][indexlon0])
#     print('Lon1 index',indexlon1)
#     print('Lon1 value',TIEGCM['lon'][indexlon1])
#     print()
#     print('Lat0 index',indexlat0)
#     print('Lat0 value',TIEGCM['lat'][indexlat0])
#     print('Lat1 index',indexlat1)
#     print('Lat1 value',TIEGCM['lat'][indexlat1])
    
#     print('alt index',indexut)
#     print('alt value',TIEGCM['ut'][indexut])

    DEN_prof00 = TIEGCM[param][indexut, :-1, indexlat0, indexlon0]
    DEN_prof10 = TIEGCM[param][indexut, :-1, indexlat1, indexlon0]
    DEN_prof01 = TIEGCM[param][indexut, :-1, indexlat0, indexlon1]
    DEN_prof11 = TIEGCM[param][indexut, :-1, indexlat1, indexlon1]
    

    ZG_prof00 = TIEGCM['ZG'][indexut, :-1, indexlat0, indexlon0]
    ZG_prof10 = TIEGCM['ZG'][indexut, :-1, indexlat1, indexlon0]
    ZG_prof01 = TIEGCM['ZG'][indexut, :-1, indexlat0, indexlon1]
    ZG_prof11 = TIEGCM['ZG'][indexut, :-1, indexlat1, indexlon1]

    DEN_df = pd.DataFrame(data = {'00' :DEN_prof00 ,\
                                  '10' :DEN_prof10 ,\
                                  '01' :DEN_prof01 ,\
                                  '11' :DEN_prof11 ,\
                                })
    ZG_df = pd.DataFrame(data = {'00' :ZG_prof00 ,\
                                  '10' :ZG_prof10 ,\
                                  '01' :ZG_prof01 ,\
                                  '11' :ZG_prof11 ,\
                                })
    return(DEN_df, ZG_df, grid_vals)



def interp_DENSITY_to_altitude(DEN_df, ZG_df, height_sat):
    '''
    Interpolation function: (4 / 5)

    This function interpolates Density in the log space then converts it back'''
    density_surf_at_height_sat = np.zeros(np.size(ZG_df.columns))
    i = 0
    for col in ZG_df.columns:
        xp = ZG_df[col][:].values*1e-5
        fp = DEN_df[col][:].values
        xval = height_sat
        density_surf_at_height_sat[i] =  np.exp( np.interp(xval, xp, np.log(fp))  )
        i += 1
    return(density_surf_at_height_sat)



def interp_param_to_altitude(df, ZG_df, height_sat):
    
    '''
    Interpolation function: (5 / 5)

    This function interpolates all other parameters (not density) using linear interpolation
    '''

    param_surf_at_height_sat = np.zeros(np.size(ZG_df.columns))
    i = 0
    for col in ZG_df.columns:
        xp = ZG_df[col][:].values*1e-5
        fp = df[col][:].values
        xval = height_sat
        param_surf_at_height_sat[i] =  np.interp(xval, xp, fp)  
        i += 1
    return(param_surf_at_height_sat)





def interpolate_tiegcm(TIEGCM, lon_sat, lat_sat, time_sat, height_sat, param):
    '''
    Interpolation function: (1 / 5)
    
    This function finds the interpolated value of a TIEGCM parameter.  
    Interpolated to a desired lon,lat,alt
    '''
    
    # Find the vertical profiles of the four grid points making up the grid square around your desired point.
#     print('time_sat:', time_sat)
    df, ZG_df, grid_vals = DEN_and_ZG_vertprofs(TIEGCM, lon_sat, lat_sat, time_sat, param)
    
    if param == 'DEN':
        param_surf_at_height_sat = interp_DENSITY_to_altitude(df, ZG_df, height_sat)
    else:
        param_surf_at_height_sat = interp_param_to_altitude(df, ZG_df, height_sat)


    xval = lon_sat

    x_lon = [grid_vals['lon0'][0], grid_vals['lon1'][0]]
    f_den0 = [param_surf_at_height_sat[0], param_surf_at_height_sat[1]]
    interp_lon0 =   np.interp(xval, x_lon, f_den0  )

    f_den1 = [param_surf_at_height_sat[2], param_surf_at_height_sat[3]]
    interp_lon1 =   np.interp(xval, x_lon, f_den1  )

    final_interped_value = interp_lon0 + ((lat_sat - grid_vals['lat0'][0]) / (grid_vals['lat1'][0] - grid_vals['lat0'][0]))*(interp_lon1 - interp_lon0)

    return(final_interped_value)



In [2]:
file_dir = '/data/data_geodyn/atmos_models_data/tiegcm/2003/' 

file = file_dir+'s319.nc'

ncdump(file , verb=True)

NetCDF Global Attributes:
	Conventions: 'CF-1.0'
	label: 'tiegcm.exe res=5.0'
	create_date: '04/24/15 22:58:26'
	logname: 'fschmidt'
	host: 'r15i0n12'
	system: 'LINUX'
	model_name: 'tiegcm'
	model_version: 'tiegcm_trunk'
	svn_version: '1133M'
	output_file: 's319.nc'
	history_type: 'secondary'
	run_type: 'continuation'
	source_file: 'p01.nc (continuation)'
	source_mtime: array([1, 0, 0], dtype=int32)
	initial_file: ''
	initial_mtime: array([0, 0, 0], dtype=int32)
	lev_to_hPa_method1: 'p0*exp(-lev(k))'
	lev_to_hPa_method2: 'p0_model*1.e-3*exp(-lev(k))'
	nhist: 24
	delhist_mins: 60
	missing_value: 1e+36
	potential_model: 'HEELIS'
	tuv_lbc_intop: 0
	contents: '03319 319  1  0 to 03320 320  0  0 by   60 tiegcm_trunk secondary'
	contents_desc: 'yyddd day hour min to yyddd day hour min by delta_mins'
NetCDF dimension information:
	Name: time
		size: 24
		type: dtype('float64')
		long_name:
		units:
		initial_year:
		initial_day:
		initial_mtime:
	Name: lon
		size: 72
		type: dtype('float64')


(['Conventions',
  'label',
  'create_date',
  'logname',
  'host',
  'system',
  'model_name',
  'model_version',
  'svn_version',
  'output_file',
  'history_type',
  'run_type',
  'source_file',
  'source_mtime',
  'initial_file',
  'initial_mtime',
  'lev_to_hPa_method1',
  'lev_to_hPa_method2',
  'nhist',
  'delhist_mins',
  'missing_value',
  'potential_model',
  'tuv_lbc_intop',
  'contents',
  'contents_desc'],
 ['time',
  'lon',
  'lat',
  'lev',
  'ilev',
  'mlon',
  'mlat',
  'mlev',
  'imlev',
  'mtimedim',
  'latlon',
  'dtidedim',
  'sdtidedim',
  'datelen',
  'filelen'],
 ['time',
  'lon',
  'lat',
  'lev',
  'ilev',
  'mlon',
  'mlat',
  'mlev',
  'imlev',
  'mtime',
  'year',
  'day',
  'calendar_advance',
  'write_date',
  'iter',
  'ntask_mpi',
  'coupled_cmit',
  'ut',
  'timestep',
  'f107d',
  'f107a',
  'hpower',
  'ctpoten',
  'Kp',
  'bximf',
  'byimf',
  'bzimf',
  'swvel',
  'swden',
  'al',
  'colfac',
  'joulefac',
  'dtide',
  'sdtide',
  'ncep_ncfile',
  

In [3]:
from netCDF4 import Dataset
netcdf_file = Dataset('/data/data_geodyn/atmos_models_data/tiegcm/2003/'+'s319.nc')

The attributes indicate the year is 2003:

In [4]:
print(netcdf_file.getncattr('contents'))
print(netcdf_file.getncattr('contents_desc'))

03319 319  1  0 to 03320 320  0  0 by   60 tiegcm_trunk secondary
yyddd day hour min to yyddd day hour min by delta_mins


The "year" variable indicates the correct year to be 2003

In [5]:
print(netcdf_file['year'])
print(netcdf_file['year'][:])

<class 'netCDF4._netCDF4.Variable'>
int32 year(time)
    long_name: calendar year
unlimited dimensions: time
current shape = (24,)
filling on, default _FillValue of -2147483647 used
[2003 2003 2003 2003 2003 2003 2003 2003 2003 2003 2003 2003 2003 2003
 2003 2003 2003 2003 2003 2003 2003 2003 2003 2003]


The "mtime" variable is the model time.  It contains the correct model day of year and hour for each model time step.  This is the variable that I have mentioned in the past is the best to use when determining model dates.

In [6]:
print(netcdf_file['mtime'])
print(netcdf_file['mtime'][:])

<class 'netCDF4._netCDF4.Variable'>
int32 mtime(time, mtimedim)
    long_name: model times (day, hour, minute)
    units: day, hour, minute
unlimited dimensions: time
current shape = (24, 3)
filling on, default _FillValue of -2147483647 used
[[319   1   0]
 [319   2   0]
 [319   3   0]
 [319   4   0]
 [319   5   0]
 [319   6   0]
 [319   7   0]
 [319   8   0]
 [319   9   0]
 [319  10   0]
 [319  11   0]
 [319  12   0]
 [319  13   0]
 [319  14   0]
 [319  15   0]
 [319  16   0]
 [319  17   0]
 [319  18   0]
 [319  19   0]
 [319  20   0]
 [319  21   0]
 [319  22   0]
 [319  23   0]
 [320   0   0]]


In [7]:
print(netcdf_file['time'])
print()
print(netcdf_file['time'][:])


<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    long_name: time
    units: minutes since 2000-3-20 0:0:0
    initial_year: 2000
    initial_day: 80
    initial_mtime: [80  0  0]
unlimited dimensions: time
current shape = (24,)
filling on, default _FillValue of 9.969209968386869e+36 used

[344220. 344280. 344340. 344400. 344460. 344520. 344580. 344640. 344700.
 344760. 344820. 344880. 344940. 345000. 345060. 345120. 345180. 345240.
 345300. 345360. 345420. 345480. 345540. 345600.]


### Run through TIEGCM Manually and with Kamodo Kamodo 


Choose a lon/lat/ut point that is on the TIEGCM grid and will not require any interpolation

In [11]:
%load_ext autoreload
%autoreload 2

from datetime import datetime, timezone, timedelta
from numpy import array
import sys

variables =['lon','lat' ,'ut','ilev','f107d','f107a','Kp', 'O2', 'O1', 'N2', 'HE', 'DEN', 'TN', 'Z', 'ZG', 'ZGMID']

for i,val in enumerate(np.arange(0,1)):  
    kamodo_program_path = '/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/orig_flythrough/SingleSatelliteFlythrough.py'
    model     =  "TIEGCM" #'4' #'TIEGCM'
    file_dir  = '/data/data_geodyn/atmos_models_data/tiegcm/2003/'
    var_list  = "['rho']"
    dz_list   = "['0']"
    coordtype = 'GDZ'     
    coordgrid = 'sph'    
    high_res  = '20.'
    sat_time1 = "031115" #str(calls_csv['YYMMDD'][i])   
    sat_time2 = "120000" #1068897600         #str(calls_csv['HHMMSS'][i])   
    utc_time  = '1068897600'         #str(datetime.timestamp(datetime.strptime(sat_time1+sat_time2, '%y%m%d%H%M%S').replace(tzinfo=timezone.utc)))
    c1        = '100.0'         #str(calls_csv['lon'][i])    # lon
    c2        = '32.5'          #str(calls_csv['lat'][i])   # lat
    c3        = '1113.435865'           # alt
    
    print("Desired Coords: (On a Grid point so no interpolation should be done)")
    print("    Lon:",c1)
    print("    Lat:",c2)
    print("    UT :",12)
    print("    Alt:",c3)
    
    tiegcm_file = '/data/data_geodyn/atmos_models_data/tiegcm/' + '%d/' % 2003  +'s%03d.nc' % 319   
    tiegcm = read_tiegcm(tiegcm_file, variables)
    tiegcm  = tiegcm[0]

    lon  = float(c1)
    lat  = float(c2)
    alt  = float(c3)
    time_hour = datetime.strptime(sat_time1+sat_time2, '%y%m%d%H%M%S').hour
    den_TIEGCM_sat_alt = interpolate_tiegcm(tiegcm, lon, lat, time_hour, alt, 'DEN')
    print('----------------------------------------------------------------------')
    print('    Manual reading of TIEGCM: ',den_TIEGCM_sat_alt ,' g/cm^3')  

    
    sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/orig_flythrough/')
    from SingleSatelliteFlythrough import SingleModelFlythrough

    ### Prepare input to the satellite Flythrough
    if len(model)==1: 
        model = int(model)
    temp_str = var_list[1:-1].replace("'","").replace(' ','').replace('"','')
    variable_list = temp_str.split(',')   #['rho','N_n']
    if array([len(item) for item in variable_list]).max()==1:
        variable_list = array(variable_list, dtype=int)
    temp_str = dz_list[1:-1].replace("'","").replace(' ','').replace('"','')
    dz = list(array(temp_str.split(','),dtype=int))  #[1,0], later treated as a boolean
    sat_time = float(utc_time) #1426637500.0
    c1 = float(c1) #x[R_E]    or    lon[deg]
    c2 = float(c2) #y[R_E]    or    lat[deg]
    c3 = float(c3) #z[R_E]    or    radius[R_E] or altitude[km]
    coord_type = coordtype  #'SPH', 'GDZ', etc
    if len(coord_type)==1: coord_type=int(coord_type)
    coord_grid = coordgrid  #'car' or 'sph'
    if len(coord_grid)==1: coord_type=int(coord_grid)
    high_res = 20.      
 
    rho = SingleModelFlythrough(model, 
                          file_dir, 
                          variable_list,
                          dz, 
                          sat_time, 
                          c1, 
                          c2, 
                          c3, 
                          coordtype, 
                          coordgrid, 
                          high_res)
    print('    Kamodo returned RHO       ', rho['rho'], 'g/cm^3')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Desired Coords: (On a Grid point so no interpolation should be done)
    Lon: 100.0
    Lat: 32.5
    UT : 12
    Alt: 1113.435865
----------------------------------------------------------------------
    Manual reading of TIEGCM:  1.1485449579612019e-16  g/cm^3
****** Not Conducting Coordinate Change.
****** Not Conducting Coordinate Change.
    Kamodo returned RHO        8.777259658348327e-23 g/cm^3


In [9]:
index_lon  = np.where(tiegcm['lon']  ==   100  )
index_lat  = np.where(tiegcm['lat']  ==   32.5 )
index_ut   = np.where(tiegcm['ut']   ==   12.  )
index_ilev = np.where(tiegcm['ilev'] ==   3.   )

print('Lon  =', tiegcm['lon'][index_lon][0] )
print('Lat  =', tiegcm['lat'][index_lat][0])
print('ut   =', tiegcm['ut'][index_ut][0] )
print('ilev =', tiegcm['ilev'][index_ilev][0] )
print('Alt  =', tiegcm['ZG'][index_ut, index_ilev, index_lat, index_lon][0][0])

print('DEN at the above coordinates:', tiegcm['DEN'][index_ut, index_ilev, index_lat, index_lon][0][0], 'g/cm^3')

Lon  = 100.0
Lat  = 32.5
ut   = 12.0
ilev = 3.0
Alt  = 36781910.0
DEN at the above coordinates: 5.305892e-15 g/cm^3
